# Setup setting and seed

In [1]:
from multilevel_sabre import MultiLevelSabre
from pathlib import Path
from typing import Dict, List, Iterable
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import SabreLayout
from qiskit.transpiler import CouplingMap, PassManager
from IPython.display import display
import json
import time


seed = 42

# Setup circuits

In [2]:
def gather_qasm_files(paths: Iterable[str]) -> List[Path]:
    """Expand a list of files/dirs into a sorted list of .qasm files."""
    result: List[Path] = []
    for p in paths:
        path = Path(p)
        if path.is_dir():
            result.extend(sorted(path.glob("*.qasm")))
        elif path.is_file() and path.suffix.lower() == ".qasm":
            result.append(path)
        else:
            print(f"Skipping {p}: not a .qasm file or directory.")
    return sorted(set(result))

filepath = "circuits/"
qasm_files = gather_qasm_files([filepath])
print(f"Found {len(qasm_files)} .qasm files in {filepath}")

Found 12 .qasm files in circuits/


In [3]:
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap, PassManager
from multilevel_sabre import MultiLevelSabre
import time
import argparse
import json
import csv
from pathlib import Path
from typing import Dict, List, Iterable
from qiskit_ibm_runtime.fake_provider.backends.sherbrooke import FakeSherbrooke
fake_backend = FakeSherbrooke()
from util import EAGLE_COUPLING, sabre, count_swaps

In [4]:
def run_comparison_example(qasm_path: Path) -> Dict:
    """Run SABRE vs MultiLevel SABRE on a single QASM file and return metrics."""
    # Load the QASM circuit
    circuit = QuantumCircuit.from_qasm_file(str(qasm_path))

    # Create the EAGLE coupling map
    coupling_map = fake_backend.coupling_map   
    coupling_map.make_symmetric()

    # Run SABRE
    print(f"\n[{qasm_path.name}] Running SABRE...")
    t0 = time.time()
    sabre_swaps, sabre_circuit = sabre(
        circuit=circuit,
        coupling=EAGLE_COUPLING,
        number_of_trial=2500,
        random_seed=1,
    )
    sabre_time = time.time() - t0
    sabre_ops = sabre_circuit.count_ops()

    # Run MultiLevel SABRE
    print(f"[{qasm_path.name}] Running MultiLevel SABRE...")
    t0 = time.time()
    multilevel_pass = PassManager([
        MultiLevelSabre(
            coupling_graph=coupling_map,
            cycles=10,
            random_seed=1,
            coarsest_solving_trials=50,
            num_interpolation=10,
            use_initial_embedding=True,
            verbose=0,
        )
    ])
    multilevel_circuit = multilevel_pass.run(circuit)
    multilevel_time = time.time() - t0
    multilevel_ops = multilevel_circuit.count_ops()

    # Count SWAPs in MultiLevel SABRE result
    multilevel_swaps = count_swaps(multilevel_circuit)

    # Prepare metrics row (keep ops as JSON strings for CSV-friendliness)
    row = {
        "qasm_file": str(qasm_path),
        "sabre_swaps": sabre_swaps,
        "sabre_time_s": round(sabre_time, 4),
        "sabre_depth": sabre_circuit.depth(),
        "sabre_2q_depth": sabre_circuit.depth(lambda x: x.operation.num_qubits == 2),
        "sabre_size": sabre_circuit.size(),
        "sabre_ops_json": json.dumps(sabre_ops, default=int),

        "multilevel_swaps": multilevel_swaps,
        "multilevel_time_s": round(multilevel_time, 4),
        "multilevel_depth": multilevel_circuit.depth(),
        "multilevel_2q_depth": multilevel_circuit.depth(lambda x: x.operation.num_qubits == 2),
        "multilevel_size": multilevel_circuit.size(),
        "multilevel_ops_json": json.dumps(multilevel_ops, default=int),
    }
    # Derived metrics
    row["speedup"] = round(row["sabre_time_s"] / row["multilevel_time_s"], 4) if row["multilevel_time_s"] > 0 else float("inf")
    row["swap_reduction_pct"] = round(((row["sabre_swaps"] - row["multilevel_swaps"]) / row["sabre_swaps"] * 100), 3) if row["sabre_swaps"] else 0.0

    # Pretty print to terminal
    print("\nComparison Results:")
    print("SABRE:")
    print(f"  - Number of SWAPs: {row['sabre_swaps']}")
    print(f"  - Compilation time: {row['sabre_time_s']:.2f} seconds")
    print(f"  - Circuit depth: {row['sabre_depth']}")
    print(f"  - Circuit 2Q depth: {row['sabre_2q_depth']}")
    print(f"  - Circuit size: {row['sabre_size']}")
    print(f"\nMultiLevel SABRE:")
    print(f"  - Number of SWAPs: {row['multilevel_swaps']}")
    print(f"  - Compilation time: {row['multilevel_time_s']:.2f} seconds")
    print(f"  - Circuit depth: {row['multilevel_depth']}")
    print(f"  - Circuit 2Q depth: {row['multilevel_2q_depth']}")
    print(f"  - Circuit size: {row['multilevel_size']}")
    print(f"\nSpeedup: {row['speedup']:.2f}x")
    print(f"SWAP reduction: {row['swap_reduction_pct']:.1f}%")

    return row


def gather_qasm_files(paths: Iterable[str]) -> List[Path]:
    """Expand a list of files/dirs into a sorted list of .qasm files."""
    result: List[Path] = []
    for p in paths:
        path = Path(p)
        if path.is_dir():
            result.extend(sorted(path.glob("*.qasm")))
        elif path.is_file() and path.suffix.lower() == ".qasm":
            result.append(path)
        else:
            print(f"Skipping {p}: not a .qasm file or directory.")
    return sorted(set(result))


def write_csv(rows: List[Dict], out_csv: Path) -> None:
    if not rows:
        print("No rows to write.")
        return
    fieldnames = [
        "qasm_file",
        "sabre_swaps", "sabre_time_s", "sabre_depth", "sabre_2q_depth", "sabre_size", "sabre_ops_json",
        "multilevel_swaps", "multilevel_time_s", "multilevel_depth", "multilevel_2q_depth", "multilevel_size", "multilevel_ops_json",
        "speedup", "swap_reduction_pct",
        "error",
    ]
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    with out_csv.open("w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            writer.writerow({k: r.get(k, "") for k in fieldnames})
    print(f"\nWrote CSV: {out_csv}")



In [5]:
csv_file = "results/comparison_notebook.csv"

rows = []
for q in qasm_files:
    try:
        rows.append(run_comparison_example(q))
    except Exception as e:
        # Keep going; record the error in the CSV
        print(f"[{q.name}] ERROR: {e}")
        rows.append({"qasm_file": str(q), "error": repr(e)})

write_csv(rows, Path(csv_file))


[adder_n118.qasm] Running SABRE...
[adder_n118.qasm] Running MultiLevel SABRE...
{0: 10, 1: 6, 2: 2, 3: 11, 4: 13, 5: 9, 6: 7, 7: 5, 8: 3, 9: 1, 10: 32, 11: 12, 12: 8, 13: 4, 14: 0, 15: 36, 16: 31, 17: 17, 18: 16, 19: 15, 20: 14, 21: 51, 22: 30, 23: 26, 24: 22, 25: 18, 26: 50, 27: 29, 28: 27, 29: 25, 30: 23, 31: 21, 32: 19, 33: 70, 34: 49, 35: 28, 36: 24, 37: 20, 38: 74, 39: 69, 40: 55, 41: 48, 42: 35, 43: 34, 44: 33, 45: 89, 46: 68, 47: 47, 48: 43, 49: 39, 50: 88, 51: 67, 52: 46, 53: 44, 54: 42, 55: 40, 56: 38, 57: 87, 58: 66, 59: 45, 60: 41, 61: 37, 62: 93, 63: 86, 64: 73, 65: 65, 66: 54, 67: 53, 68: 52, 69: 106, 70: 85, 71: 64, 72: 60, 73: 56, 74: 105, 75: 111, 76: 84, 77: 63, 78: 61, 79: 59, 80: 57, 81: 104, 82: 83, 83: 62, 84: 58, 85: 103, 86: 92, 87: 82, 88: 72, 89: 71, 90: 102, 91: 81, 92: 77, 93: 101, 94: 80, 95: 78, 96: 76, 97: 120, 98: 100, 99: 79, 100: 75, 101: 119, 102: 110, 103: 99, 104: 91, 105: 90, 106: 118, 107: 98, 108: 94, 109: 117, 110: 97, 111: 95, 112: 116, 113: 9